In [2]:
!pip install pandas
!pip install pinecone
!pip install openai
!pip install python-dotenv

In [3]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [4]:
token= os.getenv("RUNPOD_TOKEN") 
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [5]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

## Try out embeddings

In [8]:
output = client.embeddings.create(input =["hello there!"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.050730783492326736, -0.06410026550292969, 0.06318455189466476, -0.03568248078227043, 0.035285670310258865, 0.025090675801038742, 0.0824756771326065, 0.08235358446836472, 0.01435388159006834, -0.010278935544192791, 0.003964296076446772, -0.07258592545986176, 0.017871765419840813, 0.033087946474552155, 0.033423710614442825, 0.025289081037044525, 0.012652171775698662, -0.03891801834106445, -0.11818868666887283, -0.0009643656085245311, -0.009897386655211449, 0.06617589294910431, -0.03406471386551857, -0.04355765879154205, 0.024327578023076057, -0.0016950323479250073, -0.03131755813956261, 0.015628255903720856, 0.0013773926766589284, -0.12636910378932953, 0.015605363063514233, -0.004590036813169718, 0.04706791043281555, -0.024953318759799004, 0.01255296915769577, -0.014941466972231865, 0.04450390115380287, 0.025380654260516167, -0.06782418489456177, 0.0009929818334057927, 0.022114591673016548, -0.013613675720989704, 0.03870435059070587, -0.01378155779093504, 0.008623012341558933, -0.048

In [9]:
len(embedings)

384

## Wrangle dataset

In [12]:
df=pd.read_json('products/products.jsonl',lines=True)

In [13]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [14]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [15]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [17]:
texts = df['text'].tolist()

In [18]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [19]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [20]:
output = client.embeddings.create(input = texts,model=model_name)

In [21]:
embeddings = output.data

## Push data to database

In [22]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "coffeeshop",
    "metric": "cosine",
    "host": "coffeeshop-vhoiaq5.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [24]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [25]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734799385,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 